In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline

import math

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import files
uploaded = files.upload()

Saving spam.csv to spam.csv


In [6]:
import io
spam = pd.read_csv(io.BytesIO(uploaded['spam.csv']), encoding="latin1")

In [10]:
spam.head()



,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [15]:
spam.drop(["Unnamed: 3","Unnamed: 4"], axis = "columns", inplace = True)

In [16]:
spam.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [17]:
dummies = pd.get_dummies(spam.v1)
dummies.head(4)


,ham,spam
0,1,0
1,1,0
2,0,1
3,1,0


In [18]:
spam = pd.concat([spam , dummies], axis = 'columns')
spam.head()

,v1,v2,ham,spam
0,ham,"Go until jurong point, crazy.. Available only ...",1,0
1,ham,Ok lar... Joking wif u oni...,1,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,0,1
3,ham,U dun say so early hor... U c already then say...,1,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",1,0


In [19]:
spam.drop(['ham'] , axis = "columns" , inplace = True)
spam.head(10)

,v1,v2,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0
5,spam,FreeMsg Hey there darling it's been 3 week's n...,1
6,ham,Even my brother is not like to speak with me. ...,0
7,ham,As per your request 'Melle Melle (Oru Minnamin...,0
8,spam,WINNER!! As a valued network customer you have...,1
9,spam,Had your mobile 11 months or more? U R entitle...,1


In [21]:
X = spam.v2
y = spam.spam

0    0
1    0
2    1
3    0
4    0
Name: spam, dtype: uint8

In [37]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size = 0.25 )

In [38]:
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer()
X_train_count = v.fit_transform(X_train.values)
X_train_count.toarray()[:3]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [39]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()

In [40]:
model.fit(X_train_count,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [45]:
X_test_count = v.transform(X_test)
model.score(X_test_count , y_test)*100

98.42067480258436

In [44]:
emails = ["Hey bro wassup  how you doin",
          "Free discount on shopping upto 25%",
          'Hey brother , you free this monday , hoping we could catch up ', 
          "Dear Customer,Thank you for banking with State Bank of India.Thx for INB txn of Rs.100000 frm A/c X7251 ref#IF24681179 on 20JUL20. If not done by you, forward this email from email id registered with SBI to unauthorisedtransaction@sbi.co.in to block your INB credentials. In case you have not logged in to your Internet Banking, please call our Customer Care. You may also call 1-800-111109. Download YONO @ www.yonosbi.com."]
emails_count = v.transform(emails)
model.predict(emails_count)

array([0, 1, 0, 0], dtype=uint8)

In [47]:
#Basically trying to simplify our process
# by making a pipeline of functions so that 
# hume transform wagarh khud se na likhne pade 
# jaise pipeline me define kr dia , then seedhe clf me 
# apna x_train , y_train pass kro 
# and woh predict kr denge , 
#instead of the whole long pehle xtrain ko transform kro 


from sklearn.pipeline import Pipeline

clf = Pipeline( [ 
                 ('vectorizer', CountVectorizer()),
                 ('nb' , MultinomialNB())
])


In [48]:
clf.fit(X_train , y_train)

Pipeline(memory=None,
         steps=[('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('nb',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [49]:
clf.score(X_test , y_test)

0.9842067480258435

In [50]:
clf.predict(emails)

array([0, 1, 0, 0], dtype=uint8)